# 1.この課題の目的
アンサンブル学習について理解する
以下の要件をすべて満たしていた場合、合格とします。

※Jupyter Notebookを使い課題に沿った検証や説明ができている。

# 2.アンサンブル学習
3種類のアンサンブル学習の効果を小さめのデータセットで確認していきます。

- ブレンディング
- バギング
- スタッキング

## <u>小さなデータセットの用意</u>
以前も利用した回帰のデータセットを用意します。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。

train.csvを学習用（train）8割、検証用（val）2割に分割してください。

## scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

[sklearn.linear_model.LinearRegression — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

[sklearn.svm.SVR — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)

[sklearn.tree.DecisionTreeRegressor — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)



## 使用するクラスをインポート

In [1]:
# データ操作に使用するクラス
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from statistics import mean

#　前処理で使用するクラス
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
import itertools

# 機会学習のモデルのクラス
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# 評価値として使用するクラス
from sklearn.metrics import mean_squared_error

# 図を描画する際に使用するクラス
import matplotlib.pyplot as plt

## データ読み込み

In [2]:
# train.csvをデータフレーム形式で読み込んで表示
df = pd.read_csv('train.csv') 
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# 今回の課題の説明変数としてGrLivAreaとYearBuilt の列を抽出
fe_df = df.loc[: , ['GrLivArea', 'YearBuilt']]
fe_df.head()

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


In [4]:
# 今回の課題の目的変数としてSalePrice の列を抽出
ob_df = df.loc[: , ['SalePrice']]
ob_df.head()

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


In [5]:
# train.csvをデータフレーム形式で読み込んで表示
df_test = pd.read_csv('test.csv') 
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
# テストデータから課題の説明変数としてGrLivAreaとYearBuilt の列を抽出
fe_test_df = df_test.loc[: , ['GrLivArea', 'YearBuilt']]
fe_test_df.head()

,GrLivArea,YearBuilt
0,896,1961
1,1329,1958
2,1629,1997
3,1604,1998
4,1280,1992


### train_test_splitメソッドにてデータを学習用8割,検証用2割に分割

In [7]:
X_train, X_test, y_train, y_test = train_test_split(fe_df, ob_df, test_size=0.2, random_state=0)

# 【問題1】ブレンディング
ブレンディングを実装し、単一モデルより精度があがる例を最低3つ示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを示します。

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。必ずしも単一モデルの精度が高い必要はありません。

回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

## 補足

分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

[sklearn.ensemble.VotingClassifier — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

## 考察

どういった組み合わせが良いか、どのようにすると多様なモデルが作れるかを考えてみましょう。

## 精度の比較をしやすいように以下の条件で実施
- 単一モデル→全てのハイパーパラメータはdefault値で実施。
- 前処理→実施すると宣言した場合以外は、実施しない。
- ブレンディングにて使用したモデル→変更すると指定したハイパーパラメータ以外はdefault値で実施。

## 例① 単一モデル→SVM　ブレンディング→データ標準化+線形回帰
## 結果(MSEの比較)
## →単一モデルと比較しMSEが「4301252987.25683」減少

### インスタンス作成

In [8]:
svr_model = SVR()

### 学習

In [9]:
svr_model.fit(X_train, y_train)

/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

### 予測

In [10]:
svr_pred = svr_model.predict(X_test)

### ブレンディング前の精度測定(それぞれの単一モデルをハイパーパラメータdefaultで実施)

In [11]:
svr_mse = mean_squared_error(y_test, svr_pred)

print('単一モデルのMSE値　：　{}'.format(svr_mse))

単一モデルのMSE値　：　7243319908.928937


### ブレンディング後の精度(線形回帰モデルとロジスティック回帰モデルの予測値の平均値にて精度を計算)

In [12]:
pipe_lr = Pipeline([('sc',StandardScaler()),
                                ('es',LinearRegression())])

pipe_lr.fit(X_train, y_train)

pipe_lr_pred = pipe_lr.predict(X_test)

/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [13]:
br1_mse = mean_squared_error(y_test, pipe_lr_pred)
print('ブレンディング後のMSE値　：　{}'.format(br1_mse))

ブレンディング後のMSE値　：　2942066921.672107


## 予測精度の差→4301252987.25683

In [14]:
svr_mse - br1_mse

4301252987.25683

## 例② 単一モデル→SVM　ブレンディング→線形回帰+決定木の予測値の平均値
## 結果(MSEの比較)
## →単一モデルと比較し「4738689256.48173」減少
## →ブレンディングモデル①(データ標準化+線形回帰)と比較し「437436269.2249007」減少

### ブレンディング後の精度

In [15]:
lir_model = LinearRegression()
dtr_model = DecisionTreeRegressor()

In [16]:
lir_model.fit(X_train, y_train)
dtr_model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [17]:
lir_pred = lir_model.predict(X_test)
dtr_pred = dtr_model.predict(X_test)

In [18]:
lir_pred.shape

(292, 1)

In [19]:
dtr_pred = dtr_pred.reshape(-1,1)
dtr_pred.shape

(292, 1)

In [20]:
mean_pred1 = (lir_pred + dtr_pred) / 2

In [21]:
br2_mse = mean_squared_error(y_test, mean_pred1)
print('ブレンディング後のMSE値　：　{}'.format(br2_mse))

ブレンディング後のMSE値　：　2504630652.4472065


## 単一SVMモデルとの予測精度の差→4738689256.48173

In [22]:
svr_mse - br2_mse

4738689256.48173

## ブレンディング1モデル(データ標準化+線形回帰)との予測精度の差→437436269.2249007

In [23]:
br1_mse - br2_mse

437436269.2249007

## 例③ 単一モデル→SVM　ブレンディング→SVM + ランダムフォレストの予測値に重みを付けた予測値の平均値
## 結果(MSEの比較)
### →単一モデルと比較し「437640150.5729437」減少
### →ブレンディングモデル①(データ標準化+線形回帰)と比較し「437640150.5729437」減少
### →ブレンディングモデル②(線形回帰+決定木の予測値の平均値)と比較し「203881.34804296494」減少

### ブレンディング後の精度

In [24]:
svr_model = SVR()
rfr_model = RandomForestRegressor()

In [25]:
svr_model.fit(X_train, y_train)
rfr_model.fit(X_train, y_train)

/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/ipykernel_laun

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [26]:
svr_pred = lir_model.predict(X_test)
rfr_pred = dtr_model.predict(X_test)

In [27]:
svr_mse = mean_squared_error(y_test, svr_pred)
svr_mse

2942066921.672107

In [28]:
svr_pred.shape

(292, 1)

In [29]:
rfr_pred.shape

(292,)

In [30]:
rfr_mse = mean_squared_error(y_test, rfr_pred)
rfr_mse

3163333573.1350837

In [31]:
# ブレンディングの計算を行う為に、svr_predとshapeを合わせる
rfr_pred = rfr_pred.reshape(-1,1)

In [32]:
mean_pred2 = (svr_pred * 1.2 + rfr_pred * 0.8) / 2

In [33]:
br3_mse = mean_squared_error(y_test, mean_pred2)
print('ブレンディング後のMSE値　：　{}'.format(br3_mse))

ブレンディング後のMSE値　：　2504426771.0991635


## 単一SVMモデルとの予測精度の差→437640150.5729437

In [34]:
svr_mse - br3_mse

437640150.5729437

## ブレンディング1モデル(データ標準化+線形回帰)との予測精度の差→437640150.5729437


In [35]:
br1_mse - br3_mse

437640150.5729437

## ブレンディング2モデル(線形回帰+決定木の予測値の平均値)との予測精度の差→203881.34804296494

In [36]:
br2_mse - br3_mse

203881.34804296494

# 【問題2】バギング
バギングを実装し、単一モデルより精度があがる例を最低1つ示してください。

バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ブートストラップサンプル）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

[sklearn.model_selection.train_test_split — scikit-learn 0.20.0 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。

推定結果の平均をとる部分はブースティングと同様の実装になります。

## 例 単一モデル→決定木　バギング→学習データを100個のサブセットを作成し決定木で学習
## 結果(MSEの比較)
## →問題1の単一モデルと比較しMSEが「1170050763.6482437」減少

In [37]:
# BaggingRegressorクラスを利用し、サブクラスを100個作成する
models = {
    'not bagging': DecisionTreeRegressor(random_state=0),
    'bagging': BaggingRegressor(DecisionTreeRegressor(random_state=0), n_estimators=100, random_state=0) 
}

# バギングをしない場合とバギングをした場合のMSEの表示
scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    model_pred = model.predict(X_test)
    scores[model_name] = mean_squared_error(y_test, model_pred)
    print('{} のMSE {}'.format(model_name, scores[model_name]))

not bagging のMSE 3009170128.186454


/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:397: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


bagging のMSE 1839119364.5382102


## 予測精度の差→1170050763.6482437

In [38]:
3009170128.186454 - 1839119364.5382102

1170050763.6482437

# 【問題3】スタッキング
スタッキングを実装し、単一モデルより精度があがる例を最低1つ示してください。

スタッキングの手順は以下の通りです。最低限ステージ$0$とステージ$1$があればスタッキングは成立するため、それを実装してください。まずは$k_0 = 3、M_0 = 2$程度にします。

## 学習時

（ステージ 0）

- 学習データを$K_0$個に分割する。
- 分割した内の$(K_0-1)$個をまとめて学習データ、残り１個を検証用データとする組み合わせが$K_0$個作れる。
- あるモデルのインスタンスを$K_0$個用意し、異なる学習用データを使い学習する。



## 今回はステージ0(線形回帰、決定木)とステージ1(SVM)の２層とし$k_0 = 3、M_0 = 2$で実装した
## 結果(MSEの比較)→問題1の単一モデル(SVM)と比較しMSEが「683286155.5022392」減少。

## ①データを3分割する

### 学習データを3分割するindexを確認する

In [39]:
split1_index = int(len(X_train)/3)

In [40]:
split2_index = int(len(X_train)/3)*2

In [41]:
split3_index = int(len(X_train)/3)*3

### データを3分割する

In [42]:
split1 = X_train.iloc[:split1_index, :]
split2 = X_train.iloc[split1_index:split2_index,:]
split3 = X_train.iloc[split2_index:,:]

In [43]:
split1_y = y_train.iloc[:split1_index, :]
split2_y = y_train.iloc[split1_index:split2_index,:]
split3_y = y_train.iloc[split2_index:,:]

### データを結合する

In [44]:
data1 = pd.concat([split1, split2])
data2 = pd.concat([split1, split3])
data3 = pd.concat([split2, split3])

In [45]:
y1 = pd.concat([split1_y, split2_y])
y2 = pd.concat([split1_y, split3_y])
y3 = pd.concat([split2_y, split3_y])

### ステージ0の処理

In [46]:
cls1_ins1 = LinearRegression()
cls1_ins2 = LinearRegression()
cls1_ins3 = LinearRegression()
cls2_ins1 = DecisionTreeRegressor()
cls2_ins2 = DecisionTreeRegressor()
cls2_ins3 = DecisionTreeRegressor()

In [47]:
cls1_ins1.fit(data1, y1)
cls1_ins2.fit(data2, y2)
cls1_ins3.fit(data3, y3)
cls2_ins1.fit(data1, y1)
cls2_ins2.fit(data2, y2)
cls2_ins3.fit(data3, y3)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

### 各インスタンスで予測をし、結果を結合する

In [48]:
cls1_pred = np.concatenate([cls1_ins1.predict(split3), cls1_ins2.predict(split2), cls1_ins3.predict(split1)])
cls2_pred = np.concatenate([cls2_ins1.predict(split3), cls2_ins2.predict(split2), cls2_ins3.predict(split1)])
stage0_pred = np.concatenate([cls1_pred, cls2_pred.reshape(-1,1)], 1)

### 最終ステージの処理

In [49]:
last_cls = SVR()

In [50]:
last_cls.fit(stage0_pred, y_train)

/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

### 推測(スタッキング)

In [51]:
final_pred1 = np.concatenate([cls1_ins1.predict(X_test), cls1_ins2.predict(X_test), cls1_ins3.predict(X_test)], 1)
final_pred2 = np.concatenate([cls2_ins1.predict(X_test).reshape(-1,1), cls2_ins2.predict(X_test).reshape(-1,1), cls2_ins3.predict(X_test).reshape(-1,1)], 1)
final_pred = np.concatenate([final_pred1, final_pred2],1)

In [52]:
final_pred = np.mean(final_pred, axis=1)

In [53]:
stack_score = mean_squared_error(y_test, final_pred)
print('スタッキングモデルのMSE {}'.format(stack_score))

スタッキングモデルのMSE 2258780766.169868


### 学習、予測(単一モデル)

In [54]:
single_cls1 = LinearRegression()
single_cls2 = DecisionTreeRegressor()
single_cls3 = SVR()

In [55]:
single_cls1.fit(X_train, y_train)
single_cls2.fit(X_train, y_train)
single_cls3.fit(X_train, y_train)

/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/takahiro/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [56]:
single_score = {}
single_score['LinearRegression'] = mean_squared_error(y_test, single_cls1.predict(X_test))
single_score['DecisionTreeRegressor'] = mean_squared_error(y_test, single_cls2.predict(X_test))
single_score['SVR'] = mean_squared_error(y_test, single_cls3.predict(X_test))

for name, score in single_score.items():
    print('{}のMSE {}'.format(name, score))
print('単一モデルの中で最もMSEの低いモデルは{}\n'.format(min(single_score, key=single_score.get)))

LinearRegressionのMSE 2942066921.672107
DecisionTreeRegressorのMSE 3091027269.5083714
SVRのMSE 7243319908.928937
単一モデルの中で最もMSEの低いモデルはLinearRegression



In [57]:
stack_score < single_score['LinearRegression']

True

## 予測精度の差(単一モデルの中で最もMSEが低いモデルとの比較)→683286155.5022392

In [58]:
single_score['LinearRegression'] - stack_score

683286155.5022392